In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\jager\Desktop\github\LLM_agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
prompt_template = PromptTemplate.from_template(template=open("./prompt_template/ENTITIES_RELATIONS_GENERATION/ENTITIES_RELATIONS_GENERATION.md", "r").read())

In [10]:
model = ChatGoogleGenerativeAI(api_key = os.getenv("GOOGLE_API_KEY"), model=os.getenv("DEFAULT_AI_GEMINI_MODEL"))

response = model.invoke(prompt_template.invoke({"text": open("./example.txt", "r").read()}).to_string()).content

In [9]:
print(response)

['```', '("entity"|Sophie|Person|Sophie is the narrator of the text. She comes from Wuhan, went to Paris in winter, has a boyfriend named Noé, and a dog named Rookie.)', '("entity"|Wuhan|Location|Sophie\'s hometown.)', '("entity"|Paris|Location|City Sophie visited this winter.)', '("entity"|Winter|Date|The season when Sophie visited Paris.)', '("entity"|Noé|Person|Sophie\'s boyfriend who designed an app.)', '("entity"|Rookie|Person|Sophie\'s Shiba Inu dog.)', '', '', '("relationship"|Sophie|Wuhan|Sophie comes from Wuhan.|100)', '("relationship"|Sophie|Paris|Sophie visited Paris this winter.|95)', '("relationship"|Paris|Winter|Sophie\'s visit to Paris happened in winter.|90)', '("relationship"|Sophie|Noé|Noé is Sophie\'s boyfriend.|98)', '("relationship"|Noé|App|Noé designed the app.|100)', '("relationship"|Sophie|Rookie|Rookie is Sophie\'s dog.|98)', '', '```']


In [11]:
response = response.split("\n")

response = [row.replace("'", "\\'") for row in response]

entities = []
relations = []
for row in response:
    if "entity" in row:
        entities.append({
            "name": row.split("|")[1],
            "type": row.split("|")[2],
            "description": row.split("|")[3][:-1] # [:-1] to remove the closing parenthesis of the entity/relationship
        })
    elif "relationship" in row:
        relations.append({
            "from": row.split("|")[1],
            "to": row.split("|")[2],
            "description": row.split("|")[3],
            "strength": row.split("|")[4][:-1] # [:-1] to remove the closing parenthesis of the entity/relationship
        })

In [12]:
print(entities)

[{'name': 'Sophie', 'type': 'Person', 'description': 'Sophie is the narrator of the text. She comes from Wuhan, went to Paris this winter, has a boyfriend named Noé, and a dog named Rookie.'}, {'name': 'Wuhan', 'type': 'Location', 'description': 'A city where Sophie comes from.'}, {'name': 'Paris', 'type': 'Location', 'description': 'A city Sophie visited this winter.'}, {'name': 'Winter', 'type': 'Date', 'description': 'The season when Sophie visited Paris.'}, {'name': 'Noé', 'type': 'Person', 'description': "Sophie\\'s boyfriend who designed an app."}, {'name': 'Rookie', 'type': 'Person', 'description': "Sophie\\'s Shiba Inu dog."}]


In [17]:
from tools import query_neo4j_graph

for entity in entities:
    query_neo4j_graph(f"CREATE (a:{entity['type']} {{name: '{entity['name']}', description: '{entity['description']}'}})")

In [16]:
query_neo4j_graph(f"MATCH (n) DELETE n")

[]